In [1]:
#propably need to change to object with the identities of the immo. everything in immo class.
#change vermoegen = immo_wert - kredithoehe
#array - meaning: ...[i] = ... after i months

import json
import os
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
import numpy as np
import time
###
#PRIO TODO: 
#1) compute cycles in class 
#2) rendite






###
#LEGEND:
#abc = arrays (plottalbe) - may be monthly yearly etc
#_abc = constants calculated by the program
#_ABC = constants inputted by the user
#Abc = class





#fix discrepancy from rent changing every month vs yearly to the debt
#change to monthly plot, every 12th element is shown, nth element is the nth month, 12nth element after n years
#plot gesamtkosten
#EW Miete
#rendite
#zins is not changing, possibility of making changes after 10 y e.g.

In [2]:
input_dict = {"price": 1000000,
             "interest": 0.03,
             "rent": 3000,
             "own_pay": 0,
             "down_payment": 200000,
             "jahre_plot": 40,
             "mietentwicklung" : 1.0,
             "immopreisentwicklung" : 1.0,
             "interest cycle" : 3}

In [3]:
class Immo:
    def __init__(self):
        self.vermoegen = []
        self.kosten_ges = []
        self.immo_wert = []
        self.miete = []
        self.gewinn = []
        self.kredithoehe = []
        self._ZINS = None
        self._PREIS = None
        self._EIGENANTEIL = None
        self._ANZAHLUNG = None
        self._MIETENTWICKLUNG = None
        self._PREISENTWICKLUNG_IMMO = None
        self._ZINSZAHLUNG = None
        self._preisentwicklung_immo_jahr = None
        self._preisentwicklung_immo_monatlich = None
        self._preisentwicklung_immo_halbjahr = None
        self._preisentwicklung_miete_jahr = None
        self._preisentwicklung_miete_monatlich = None
        self._preisentwicklung_miete_halbjahr = None
        self._zins_jahr = None
        self._zins_monatlich = None
        self._zins_halbjahr = None
        #in full_compute()
        self.rendite = []
        self.pass_einkomen_prozent = []
    def debug(self):
        print(self.vermoegen)
        print(self.kosten_ges)
        print(self.immo_wert)
        print(self.miete)
        print(self.gewinn)
        print(self.kredithoehe)
        print(self._ZINS)
        print(self._PREIS)
        print(self._EIGENANTEIL)
        print(self._ANZAHLUNG)
        print(self._MIETENTWICKLUNG)
        print(self._PREISENTWICKLUNG_IMMO)
        print(self._ZINSZAHLUNG)
        print(self._preisentwicklung_immo_jahr)
        print(self._preisentwicklung_immo_monatlich)
        print(self._preisentwicklung_immo_halbjahr)
        print(self._preisentwicklung_miete_jahr)
        print(self._preisentwicklung_miete_monatlich)
        print(self._preisentwicklung_miete_halbjahr)
        print(self._zins_jahr)
        print(self._zins_monatlich)
        print(self._zins_halbjahr)
    
        
        
        
    def setData(dic):
        #input is a dictionary
        pass
        
        
    def compute_cycle_month(self):
        #compute with given data, then change data for the new computation
        if len(self.vermoegen)==0: #setting starting values at t0
            self.immo_wert.append(self._PREIS)
            self.kosten_ges.append(self._PREIS)
            self.gewinn.append(0)
            self.vermoegen.append(self._ANZAHLUNG)
            self.kredithoehe.append(self._PREIS - self._ANZAHLUNG)
            self.miete.append(textbox_miete.value)
            #miete has been already added its value upon input of new values - miete[i] = miete für den i+1 ten monat
            return
        if self.kredithoehe[-1] > self.miete[-1] + self._EIGENANTEIL: #kann nicht komplett getilgt werde
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_monatlich)
            assert(len(self.kredithoehe)==len(self.miete))
            self.kredithoehe.append((self.kredithoehe[-1]-self.miete[-1]-self._EIGENANTEIL)*(self._zins_monatlich))
            self.kosten_ges.append(self.kosten_ges[-1]+(self.kredithoehe[-1]-(self.kredithoehe[-2]-self.miete[-1]-self._EIGENANTEIL)))
            self.gewinn.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1]-self.immo_wert[-2])+self.kredithoehe[-2]-self.kredithoehe[-1])
        elif self.kredithoehe[-1] > 0:
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_monatlich)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.miete[-1] + self._EIGENANTEIL - self.kredithoehe[-1])
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1])
            self.kredithoehe.append(0)
        elif self.kredithoehe[-1] < 0: #ERROR NEGATIVER KREDIT
            print("Error: negativer Kreditwert")
        else: #KREDIT-FREI
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_monatlich)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.gewinn[-1] + self.miete[-1])
            self.kredithoehe.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1])
        self.miete.append(self.miete[-1] * self._preisentwicklung_miete_monatlich)

    def compute_cycle_half(self):
        self.debug()
        #compute with given data, then change data for the new computation
        if len(self.vermoegen)==0: #setting starting values at t0
            self.immo_wert.append(self._PREIS)
            self.kosten_ges.append(self._PREIS)
            self.gewinn.append(0)
            self.vermoegen.append(self._ANZAHLUNG)
            self.kredithoehe.append(self._PREIS - self._ANZAHLUNG)
            self.miete.append(textbox_miete.value)
            #miete has been already added its value upon input of new values - miete[i] = miete für den i+1 ten monat
            return
        if self.kredithoehe[-1] > self.miete[-1]*6 + self._EIGENANTEIL*6: #kann nicht komplett getilgt werde
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_halbjahr)
            assert(len(self.kredithoehe)==len(self.miete))
            self.kredithoehe.append((self.kredithoehe[-1]-self.miete[-1]*6-self._EIGENANTEIL*6)*(self._zins_halbjahr))
            self.kosten_ges.append(self.kosten_ges[-1]+(self.kredithoehe[-1]-(self.kredithoehe[-2]-self.miete[-1]*6-self._EIGENANTEIL*6)))
            self.gewinn.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1]-self.immo_wert[-2])+self.kredithoehe[-2]-self.kredithoehe[-1])
        elif self.kredithoehe[-1] > 0:
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_halbjahr)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.miete[-1]*6 + self._EIGENANTEIL *6 - self.kredithoehe[-1])
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1]*6)
            self.kredithoehe.append(0)
        elif self.kredithoehe[-1] < 0: #ERROR NEGATIVER KREDIT
            print("Error: negativer Kreditwert")
        else: #KREDIT-FREI
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_halbjahr)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.gewinn[-1] + self.miete[-1]*6)
            self.kredithoehe.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1]*6)
        self.miete.append(self.miete[-1] * self._preisentwicklung_miete_halbjahr)


    def compute_cycle_year(self):
        #compute with given data, then change data for the new computation
        if len(self.vermoegen)==0: #setting starting values at t0
            self.immo_wert.append(self._PREIS)
            self.kosten_ges.append(self._PREIS)
            self.gewinn.append(0)
            self.vermoegen.append(self._ANZAHLUNG)
            self.kredithoehe.append(self._PREIS - self._ANZAHLUNG)
            self.miete.append(textbox_miete.value)
            #miete has been already added its value upon input of new values - miete[i] = miete für den i+1 ten monat
            return
        if self.kredithoehe[-1] > self.miete[-1]*12 + self._EIGENANTEIL*12: #kann nicht komplett getilgt werde
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_jahr)
            assert(len(self.kredithoehe)==len(self.miete))
            self.kredithoehe.append((self.kredithoehe[-1]-self.miete[-1]*12-self._EIGENANTEIL*12)*(self._zins_jahr))
            self.kosten_ges.append(self.kosten_ges[-1]+(self.kredithoehe[-1]-(self.kredithoehe[-2]-self.miete[-1]*12-self._EIGENANTEIL*12)))
            self.gewinn.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1]-self.immo_wert[-2])+self.kredithoehe[-2]-self.kredithoehe[-1])
        elif self.kredithoehe[-1] > 0:
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_jahr)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.miete[-1]*12 + self._EIGENANTEIL *12 - self.kredithoehe[-1])
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1]*12)
            self.kredithoehe.append(0)
        elif self.kredithoehe[-1] < 0: #ERROR NEGATIVER KREDIT
            print("Error: negativer Kreditwert")
        else: #KREDIT-FREI
            self.immo_wert.append(self.immo_wert[-1] * self._preisentwicklung_immo_jahr)
            self.kosten_ges.append(self.kosten_ges[-1])
            self.gewinn.append(self.gewinn[-1] + self.miete[-1]*12)
            self.kredithoehe.append(0)
            self.vermoegen.append(self.vermoegen[-1]+(self.immo_wert[-1] - self.immo_wert[-2]) + self.miete[-1]*12)
        self.miete.append(self.miete[-1] * self._preisentwicklung_miete_jahr)
                
    def standardize(self):
        immo_wert_temp = []
        kosten_ges_temp = []
        gewinn_temp = []                       
        vermoegen_temp = [] 
        kredithoehe_temp = [] 
        if self._ZINSZAHLUNG==1: #monthly
            for j in range(len(self.gewinn)):
                if j%12==0:
                    immo_wert_temp.append(self.immo_wert[j])
                    kosten_ges_temp.append(self.kosten_ges[j])
                    gewinn_temp.append(self.gewinn[j])
                    vermoegen_temp.append(self.vermoegen[j])
                    kredithoehe_temp.append(self.kredithoehe[j])
        elif self._ZINSZAHLUNG==2: #half
            pass
        elif self._ZINSZAHLUNG==3: #yearly
            return
        self.immo_wert = immo_wert_temp
        self.kosten_ges = kosten_ges_temp
        self.gewinn = gewinn_temp                       
        self.vermoegen = vermoegen_temp 
        self.kredithoehe = kredithoehe_temp 


In [4]:
def full_compute(ak):
    immo = ak
    if immo._ZINSZAHLUNG==1:
        for i in range(slider_jahre_plot.value*12):
            immo.compute_cycle_month()
    elif immo._ZINSZAHLUNG==2:
        for i in range(slider_jahre_plot.value*2):
            immo.compute_cycle_half()
    elif immo._ZINSZAHLUNG==3:
        for i in range(slider_jahre_plot.value):
            immo.compute_cycle_year()    
    immo.standardize()

In [5]:
def get_rendite(vermoegen):
    basis = vermoegen[0]
    rendite = []
    for i in range(len(vermoegen)-1):
        diff = vermoegen[i+1] - vermoegen[i]
        rendite.append(diff/basis)
    return rendite

In [6]:
def computeandplot(i):
    
    immo = Immo() #creating new immo object
    #initialize immo "global" variables from input of the user
    immo._ZINS = textbox_zins.value
    immo._PREIS = textbox_price.value
    immo._EIGENANTEIL = textbox_eigenanteil.value
    immo._ANZAHLUNG = textbox_anzahlung.value
    immo._MIETENTWICKLUNG = slider_entwicklung_miete.value/100 +1
    immo._PREISENTWICKLUNG_IMMO = slider_entwicklung_immo.value/100 +1
    immo._ZINSZAHLUNG = selector_zinskosten.value
    
    #compute "local" variables for immo

    
    immo._preisentwicklung_immo_jahr = immo._PREISENTWICKLUNG_IMMO
    immo._preisentwicklung_immo_monatlich = immo._preisentwicklung_immo_jahr ** (1/12)
    immo._preisentwicklung_immo_halbjahr = immo._preisentwicklung_immo_jahr ** (1/2)
    immo._preisentwicklung_miete_jahr = immo._MIETENTWICKLUNG
    immo._preisentwicklung_miete_monatlich = immo._preisentwicklung_miete_jahr ** (1/12)
    immo._preisentwicklung_miete_halbjahr = immo._preisentwicklung_miete_jahr ** (1/2)
    immo._zins_jahr = immo._ZINS +1
    immo._zins_monatlich = immo._zins_jahr ** (1/12)
    immo._zins_halbjahr = immo._zins_jahr ** (1/2)
    
    
    full_compute(immo)
    
    immo.rendite = get_rendite(immo.vermoegen)
    figure_gewinn_kredit.update_traces(y=immo.gewinn, selector=dict(name="Gewinn"))
    figure_gewinn_kredit.update_traces(y=immo.kredithoehe, selector=dict(name="Kredit"))
    figure_vermoegen.update_traces(y=immo.vermoegen, selector=dict(name="Vermögen"))
    figure_kosten_ges.update_traces(y=immo.kosten_ges, selector=dict(name="Kosten"))
    figure_rendite.update_traces(y=immo.rendite, selector=dict(name="Rendite"))
    return immo
    

In [7]:

def computeandexport(i):
    

    immo = Immo() #creating new immo object

    immo._ZINS = textbox_zins.value
    immo._PREIS = textbox_price.value
    immo._EIGENANTEIL = textbox_eigenanteil.value
    immo._ANZAHLUNG = textbox_anzahlung.value
    immo._MIETENTWICKLUNG = slider_entwicklung_miete.value/100 +1
    immo._PREISENTWICKLUNG_IMMO = slider_entwicklung_immo.value/100 +1
    immo._ZINSZAHLUNG = selector_zinskosten.value
    
    #compute "local" variables for immo

    
    immo._preisentwicklung_immo_jahr = immo._PREISENTWICKLUNG_IMMO
    immo._preisentwicklung_immo_monatlich = immo._preisentwicklung_immo_jahr ** (1/12)
    immo._preisentwicklung_immo_halbjahr = immo._preisentwicklung_immo_jahr ** (1/2)
    immo._preisentwicklung_miete_jahr = immo._MIETENTWICKLUNG
    immo._preisentwicklung_miete_monatlich = immo._preisentwicklung_miete_jahr ** (1/12)
    immo._preisentwicklung_miete_halbjahr = immo._preisentwicklung_miete_jahr ** (1/2)
    immo._zins_jahr = immo._ZINS +1
    immo._zins_monatlich = immo._zins_jahr ** (1/12)
    immo._zins_halbjahr = immo._zins_jahr ** (1/2)
    
    
    full_compute(immo)
    immo.rendite = get_rendite(immo.vermoegen)
    ex_dict = {"ZINS": immo._ZINS,
        "PREIS": immo._PREIS,
        "EIGENANTEIL": immo._EIGENANTEIL,
        "ANZAHLUNG": immo._ANZAHLUNG,
        "MIETENTWICKLUNG": immo._MIETENTWICKLUNG,
        "PREISENTWICKLUNG_IMMO": immo._PREISENTWICKLUNG_IMMO,
        "ZINSZAHLUNG": immo._ZINSZAHLUNG,
        "vermoegen": immo.vermoegen,
        "kosten_ges": immo.kosten_ges,
        "immo_wert": immo.immo_wert,
        "miete": immo.miete,
        "gewinn": immo.gewinn,
        "kredithoehe": immo.kredithoehe,
        "rendite": immo.rendite}

    time_now = int(time.time())
    os.system(f"cd /Users/a2/.cache/immocalc && mkdir {time_now} && cd {time_now} && touch parameters.json")
    with open(f"/Users/a2/.cache/immocalc/{time_now}/parameters.json", "w") as f:
        json.dump(ex_dict, f)
    
    #create plots
    figure_gewinn_kredit = go.Figure().add_trace(go.Scatter(y=immo.gewinn, mode="lines", name = "Gewinn")).add_trace(go.Scatter(y=immo.kredithoehe, mode="lines", name = "Kredit")).update_layout(title_text="Rentabilität", title_x=0.5)
    figure_kosten_ges = go.Figure().add_trace(go.Scatter(y=immo.kosten_ges, mode="lines", name="Kosten")).update_layout(title_text = "Gesamtkosten", title_x=0.5)
    figure_vermoegen = go.Figure().add_trace(go.Scatter(y=immo.vermoegen, mode="lines", name="Vermögen")).update_layout(title_text="Vermögen", title_x=0.5)
    figure_rendite = go.Figure().add_trace(go.Scatter(y=immo.rendite, mode="lines", name="Rendite")).update_layout(title_text="Rendite", title_x=0.5)
    figure_gewinn_kredit.write_html(f"/Users/a2/.cache/immocalc/{time_now}/gk.html")
    figure_kosten_ges.write_html(f"/Users/a2/.cache/immocalc/{time_now}/kg.html")
    figure_vermoegen.write_html(f"/Users/a2/.cache/immocalc/{time_now}/vm.html")
    figure_rendite.write_html(f"/Users/a2/.cache/immocalc/{time_now}/re.html")

    

    

In [8]:
if __name__ == "__main__":
    

    ##########
    #INIT INPUT WIDGETS
    ##########

    slider_entwicklung_immo = widgets.FloatSlider(min=-5, max=7, description='EW Immo: ', value=0)
    slider_entwicklung_miete = widgets.FloatSlider(min=-5, max=7, description='EW Miete: ', value=0)
    slider_jahre_plot = widgets.IntSlider(min=5, max=100, description='Jahre: ', value=input_dict["jahre_plot"])


    textbox_zins = widgets.FloatText(value=input_dict["interest"], min=0, max=15.0, step=0.0001, description='Zins:', disabled=False)
    textbox_price = widgets.IntText(value=input_dict["price"], description='Preis:   ', disabled=False)
    textbox_miete = widgets.IntText(value=input_dict["rent"], description='Miete:   ', disabled=False)
    textbox_anzahlung = widgets.IntText(value=input_dict["down_payment"], description='Anzahlung:   ', disabled=False)
    textbox_eigenanteil = widgets.IntText(value=input_dict["own_pay"], description='Eigenanteil:   ', disabled=False)

    selector_zinskosten = widgets.Dropdown(
        options=[('monatlich', 1), ('halbjährlich', 2), ('jährlich', 3)],
        value=3,
        description='Zinszahlung:'
    )
    button_compute = widgets.Button(value=False, description='Compute', disabled=False)
    button_export = widgets.Button(value=False, description='Export', disabled=False)#export now input also multiple cases

    #########
    #ADDING THE CONTAINERS FOR INPUT
    #########
    
    container1 = widgets.HBox([textbox_price, slider_jahre_plot])
    container2 = widgets.HBox([textbox_zins, textbox_anzahlung, selector_zinskosten])
    container3 = widgets.HBox([textbox_miete, textbox_eigenanteil])
    container4 = widgets.HBox([slider_entwicklung_immo, slider_entwicklung_miete, button_compute, button_export])
    container_inp = widgets.VBox([container1, container2, container3, container4])

    display(container_inp)


    ##########
    #OBSERVE CHANGES IN INPUT WIDGETS
    ##########

    immo = slider_entwicklung_immo.observe(computeandplot)
    immo = slider_entwicklung_miete.observe(computeandplot)
    immo = slider_jahre_plot.observe(computeandplot)
    immo = button_compute.on_click(computeandplot)
    def export_helper(i):
        print("starting")
        computeandexport(immo)
        print("exported!?")
    button_export.on_click(export_helper)
    

    #########
    #INTRODUCING FIGURES
    #########

    figure_gewinn_kredit = go.FigureWidget()
    figure_vermoegen = go.FigureWidget()
    figure_kosten_ges = go.FigureWidget()
    figure_rendite = go.FigureWidget()

    figure_gewinn_kredit.add_trace(go.Scatter(y=[], mode="lines", name = "Gewinn"))
    figure_gewinn_kredit.add_trace(go.Scatter(y=[], mode="lines", name = "Kredit"))
    figure_gewinn_kredit.update_layout(title_text="Rentabilität", title_x=0.5)

    figure_vermoegen.add_trace(go.Scatter(y=[], mode="lines", name = "Vermögen"))
    figure_vermoegen.update_layout(title_text="Vermögen", title_x=0.5)
    
    figure_kosten_ges.add_trace(go.Scatter(y=[], mode="lines", name="Kosten"))
    figure_kosten_ges.update_layout(title_text = "Gesamtkosten", title_x=0.5)
    
    figure_rendite.add_trace(go.Scatter(y=[], mode="lines", name="Rendite"))
    figure_rendite.update_layout(title_text="Rendite", title_x=0.5)


    container5 = widgets.HBox([figure_gewinn_kredit, figure_vermoegen])
    container6 = widgets.HBox([figure_kosten_ges])
    container7 = widgets.HBox([figure_rendite])
    container_fig = widgets.VBox([container5, container6, container7])
    display(container_fig)





    'data': [{'mode': 'lines',
              'name': 'Gewinn',
  …